# Lean 7 - Integration des LLMs pour l'Assistance aux Preuves

**Navigation** : [← Lean-6-Mathlib-Essentials](Lean-6-Mathlib-Essentials.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-8-Agentic-Proving →](Lean-8-Agentic-Proving.ipynb)

---


## Introduction

L'annee 2024-2026 a marque un tournant decisif dans l'histoire des mathematiques formelles. Les **Large Language Models** (LLMs) ont commence a prouver des theoremes de maniere autonome, avec des succes spectaculaires qui ont bouleverse la communaute mathematique :

- **AlphaProof** (DeepMind) : Medaille d'argent aux Olympiades Internationales de Mathematiques 2024, publie dans Nature en novembre 2025
- **Harmonic Aristotle** : Medaille d'or IMO 2025, resolution de 15+ problemes d'Erdos depuis Noel 2025, dont le #124 variant (~30 ans) en 6h
- **DeepSeek-Prover** : Resolution de multiples problemes d'Erdos (379, 987, 730, 198)
- **LeanCopilot** (LeanDojo) : Automatisation de 74.2% des etapes de preuves Mathlib, papier NeurIPS 2025
- **LeanAgent** : Apprentissage lifelong pour theorem proving, papier ICLR 2025

Ce notebook explore comment utiliser les LLMs pour accelerer et assister la construction de preuves Lean, en s'appuyant sur ces avancees recentes.

### Objectifs pedagogiques

1. Comprendre les percees recentes en theorem proving assiste par LLM
2. Decouvrir LeanCopilot, LeanProgress et l'ecosysteme LeanDojo
3. Maitriser les patterns de collaboration humain-LLM-Lean
4. Experimenter le prompting efficace pour generer des preuves
5. Comprendre les architectures d'AlphaProof, APOLLO et LeanAgent

### Prerequis

- Notebooks **Lean-1** a **Lean-6** completes
- Cle API OpenAI ou Anthropic (optionnel pour les exercices pratiques)

### Duree estimee : 50-55 minutes

---

## L'Ere des LLMs en Mathematiques Formelles

### Timeline des percees majeures

| Date | Systeme | Accomplissement |
|------|---------|----------------|
| Juillet 2024 | AlphaProof | Medaille d'argent IMO 2024 (4 problemes sur 6, 28/42 points) |
| Octobre 2024 | LeanCopilot | Papier NeurIPS 2025 accepte, 74.2% automatisation Mathlib |
| Novembre 2025 | AlphaProof | Publication dans Nature, details sur 100M problemes d'entrainement |
| Decembre 2025 | Harmonic Aristotle | Resolution 15+ problemes Erdos depuis Noel |
| Janvier 2026 | LeanAgent | Papier ICLR 2025 sur apprentissage lifelong |
| Janvier 2026 | LeanProgress | Papier TMLR 2025 sur prediction de progression |
| Janvier 2026 | Lean4Lean | Presentation POPL 2026, bootstrap de Lean en Lean |
| Juillet 2025 | Harmonic Aristotle | Medaille d'or IMO 2025 |

## 1. AlphaProof : L'Architecture de DeepMind

### 1.1 Vue d'ensemble (Nature, Novembre 2025)

AlphaProof est le premier systeme d'IA a atteindre le niveau medaille aux Olympiades Internationales de Mathematiques. Publie dans **Nature** en novembre 2025, il combine plusieurs innovations :

1. **Fine-tuning de Gemini** sur des preuves formelles Lean
2. **Apprentissage par renforcement (AlphaZero-style)** pour guider la recherche de preuves
3. **Generation de 100 millions de theoremes synthetiques** pour l'entrainement
4. **Verification formelle systematique** avec Lean comme oracle de verite

### 1.2 Resultats IMO 2024

| Probleme | Difficulte | Points | Temps |
|----------|------------|--------|-------|
| P1 | Facile | 7/7 | Minutes |
| P2 | Moyen | 7/7 | Heures |
| P3 | Difficile | 0/7 | Timeout |
| P4 | Moyen | 7/7 | Heures |
| P5 | Difficile | 0/7 | Non resolu |
| P6 | Tres difficile | 7/7 | 3 jours |
| **Total** | | **28/42** | Medaille Argent |

**Note** : P6, le probleme le plus difficile (seulement 5 participants humains l'ont resolu), a ete resolu en 3 jours par AlphaProof.

### 1.3 Le cycle AlphaProof

```
+---------------------------------------------------------------+
|                    Cycle AlphaProof                           |
+---------------------------------------------------------------+
|                                                               |
|  Enonce (langage naturel)                                     |
|          |                                                    |
|          v                                                    |
|  Formalisation en Lean  <-----------------+                   |
|  (modele Gemini fine-tune)                |                   |
|          |                                |                   |
|          v                                |                   |
|  Generation de tactiques (LLM + MCTS)     |                   |
|  - Recherche guidee par valeur            |                   |
|  - Exploration-exploitation balance       |                   |
|          |                                |                   |
|          v                                |                   |
|  Verification Lean ------> Succes --------+----> Preuve       |
|          |                                                    |
|          v                                                    |
|       Echec ----------------------------------------+         |
|   (feedback erreur Lean)                            |         |
|          |                                          |         |
|          v                                          v         |
|   Apprentissage par renforcement        Nouvelle tentative    |
|   (mise a jour politique)                                     |
|                                                               |
+---------------------------------------------------------------+
```

### 1.4 Details techniques (du papier Nature)

- **Modele de base** : Gemini fine-tune sur ~1M preuves formelles existantes
- **Self-play** : Generation de 100M problemes synthetiques avec preuves
- **Recherche** : Monte Carlo Tree Search (MCTS) guide par un reseau de valeur
- **Hardware** : TPU v4 clusters pour l'entrainement, evaluation en parallele

In [1]:
# Exemple conceptuel du flux AlphaProof
# (Ce code illustre le principe, pas l'implementation reelle)

class AlphaProofConcept:
    """Illustration conceptuelle de l'architecture AlphaProof."""
    
    def __init__(self, llm, lean_verifier):
        self.llm = llm
        self.lean = lean_verifier
        self.max_attempts = 1000
    
    def prove(self, theorem_statement: str) -> str:
        """Tente de prouver un theoreme."""
        # Etape 1: Formaliser l'enonce
        formal_statement = self.formalize(theorem_statement)
        
        for attempt in range(self.max_attempts):
            # Etape 2: Generer des tactiques
            tactics = self.llm.generate_tactics(formal_statement)
            
            # Etape 3: Verifier avec Lean
            result = self.lean.verify(formal_statement, tactics)
            
            if result.success:
                return tactics
            
            # Etape 4: Utiliser le feedback pour ameliorer
            self.llm.learn_from_error(result.error_message)
        
        raise ProofNotFound("Limite de tentatives atteinte")

print("Architecture AlphaProof illustree")

Architecture AlphaProof illustree


## 2. LeanCopilot et l'Ecosysteme LeanDojo

### 2.1 Presentation

**LeanDojo** (https://leandojo.org) est un ecosysteme complet pour le machine learning sur les preuves Lean, developpe par une equipe de recherche incluant Caltech, Stanford et MIT. Il comprend plusieurs composants :

| Outil | Description | Publication |
|-------|-------------|-------------|
| **LeanDojo** | Framework d'extraction de donnees et interaction avec Lean | NeurIPS 2023 |
| **LeanCopilot** | Copilote LLM integre dans Lean/VS Code | NeurIPS 2025 |
| **LeanProgress** | Prediction du nombre d'etapes restantes | TMLR 2025 |
| **LeanAgent** | Apprentissage lifelong pour theorem proving | ICLR 2025 |

### 2.2 LeanCopilot (NeurIPS 2025)

**LeanCopilot** (https://github.com/lean-dojo/LeanCopilot) integre des LLMs directement dans Lean 4 pour suggerer des tactiques en temps reel.

**Resultats cles** :
- **74.2%** des etapes de preuves Mathlib peuvent etre automatisees
- Integration transparente avec VS Code
- Modeles locaux (Llama) ou API (GPT-4, Claude)

| Fonctionnalite | Description | Exemple |
|----------------|-------------|---------|
| `suggest_tactics` | Suggere des tactiques pour le but courant | `by suggest_tactics` |
| `search_proofs` | Recherche complete de preuves | `by search_proofs` |
| `select_premises` | Selectionne les lemmes pertinents | Filtrage intelligent |

### 2.3 LeanProgress (TMLR 2025)

**LeanProgress** predit combien d'etapes il reste pour completer une preuve, permettant de guider la recherche plus efficacement.

- Entraine sur des traces de preuves Mathlib
- Predit le "progress" (0-1) vers la completion
- Utilise comme heuristique dans la recherche de preuves

### 2.4 LeanAgent (ICLR 2025)

**LeanAgent** introduit l'apprentissage continu (lifelong learning) pour le theorem proving :

- **Curriculum automatique** : Commence par des theoremes simples, progresse vers les difficiles
- **Memoire des preuves passees** : Reutilise les patterns appris
- **Adaptation dynamique** : S'ameliore au fil des interactions

### 2.5 Architecture LeanDojo

```
+------------------------------------------------------------------+
|                      Ecosysteme LeanDojo                         |
+------------------------------------------------------------------+
|                                                                  |
|  +----------------+    +------------------+    +---------------+ |
|  |    Lean 4      |<-->|    LeanDojo      |<-->|   ML Models   | |
|  | (verificateur) |    | (extraction data)|    | (ReProver,etc)| |
|  +----------------+    +------------------+    +---------------+ |
|         ^                      |                      |          |
|         |                      v                      v          |
|         |              +------------------+    +---------------+ |
|         |              |   LeanCopilot    |    | LeanProgress  | |
|         +------------->|  (suggestions)   |    | (prediction)  | |
|                        +------------------+    +---------------+ |
|                                                                  |
|                        +------------------+                      |
|                        |    LeanAgent     |                      |
|                        | (lifelong learn) |                      |
|                        +------------------+                      |
|                                                                  |
+------------------------------------------------------------------+
```

In [2]:
# Exemple d'utilisation de LeanCopilot (necessite installation)

LEANCOPILOT_EXAMPLE = """
-- import LeanCopilot

-- theorem example_copilot (n : Nat) : n + 0 = n := by
--   suggest_tactics  -- LeanCopilot suggere: rfl, simp, exact Nat.add_zero n

-- theorem harder_example (a b c : Nat) : (a + b) + c = a + (b + c) := by
--   suggest_tactics  -- Suggere: exact Nat.add_assoc a b c

-- Sans LeanCopilot, on fait manuellement:
theorem manual_example (n : Nat) : n + 0 = n := by
  rfl  -- ou exact Nat.add_zero n
"""

print("Exemple LeanCopilot (code Lean):")
print(LEANCOPILOT_EXAMPLE)

Exemple LeanCopilot (code Lean):

-- import LeanCopilot

-- theorem example_copilot (n : Nat) : n + 0 = n := by
--   suggest_tactics  -- LeanCopilot suggere: rfl, simp, exact Nat.add_zero n

-- theorem harder_example (a b c : Nat) : (a + b) + c = a + (b + c) := by
--   suggest_tactics  -- Suggere: exact Nat.add_assoc a b c

-- Sans LeanCopilot, on fait manuellement:
theorem manual_example (n : Nat) : n + 0 = n := by
  rfl  -- ou exact Nat.add_zero n



### 2.3 Installation de LeanCopilot

LeanCopilot s'installe via Lake en ajoutant la dependance au `lakefile.lean`. Il necessite un modele LLM (local ou API) et s'integre dans VS Code pour les suggestions en temps reel.

In [3]:
# Dans lakefile.lean de votre projet:

# require LeanCopilot from git
#   "https://github.com/lean-dojo/LeanCopilot.git"

# Puis:
# lake update
# lake build

# Configuration de l'API (dans .env ou variable d'environnement)
# OPENAI_API_KEY=sk-...

## 3. Patterns de Collaboration Humain-LLM-Lean

### 3.1 "Vibe Coding" avec ChatGPT/Claude

L'approche la plus simple : utiliser un LLM conversationnel pour esquisser des preuves.

In [4]:
# Exemple de prompt pour "vibe coding" avec un LLM

VIBE_CODING_PROMPT = """
Je travaille sur une preuve Lean 4. Voici mon theoreme:

```lean
theorem my_theorem (a b : Nat) : a + b = b + a := by
  sorry
```

Comment puis-je completer cette preuve? 
Donne-moi le code Lean exact avec les tactiques appropriees.
"""

# Reponse typique du LLM:
LLM_RESPONSE = """
Pour prouver la commutativite de l'addition sur Nat, vous pouvez utiliser:

```lean
theorem my_theorem (a b : Nat) : a + b = b + a := by
  exact Nat.add_comm a b
```

Ou avec une preuve plus detaillee par recurrence:

```lean
theorem my_theorem (a b : Nat) : a + b = b + a := by
  induction b with
  | zero => simp [Nat.add_zero, Nat.zero_add]
  | succ n ih => simp [Nat.add_succ, Nat.succ_add, ih]
```
"""

print("Exemple de vibe coding:")
print(VIBE_CODING_PROMPT[:100] + "...")

Exemple de vibe coding:

Je travaille sur une preuve Lean 4. Voici mon theoreme:

```lean
theorem my_theorem (a b : Nat) : a...


### 3.2 Proof Sketching

Le proof sketching consiste a utiliser le LLM pour generer la structure d'une preuve, puis a completer les details manuellement. Le LLM excelle pour identifier les etapes cles.

In [5]:
# Exemple de proof sketch genere par LLM

PROOF_SKETCH_EXAMPLE = """
-- LLM genere la structure:
theorem distributivity (a b c : Nat) : a * (b + c) = a * b + a * c := by
  -- Etape 1: Recurrence sur c (suggere par LLM)
  induction c with
  | zero => 
    -- Cas de base: a * (b + 0) = a * b + a * 0
    simp   -- Humain complete
  | succ n ih =>
    -- Cas inductif: utiliser l'hypothese de recurrence
    -- (Details a completer par l'humain)
    simp [Nat.add_succ, Nat.mul_succ]
    omega  -- ou linarith avec Mathlib
"""

print("Exemple de proof sketch (code Lean):")
print(PROOF_SKETCH_EXAMPLE)

Exemple de proof sketch (code Lean):

-- LLM genere la structure:
theorem distributivity (a b c : Nat) : a * (b + c) = a * b + a * c := by
  -- Etape 1: Recurrence sur c (suggere par LLM)
  induction c with
  | zero => 
    -- Cas de base: a * (b + 0) = a * b + a * 0
    simp   -- Humain complete
  | succ n ih =>
    -- Cas inductif: utiliser l'hypothese de recurrence
    -- (Details a completer par l'humain)
    simp [Nat.add_succ, Nat.mul_succ]
    omega  -- ou linarith avec Mathlib



### 3.3 DeepAlgebra Loop

La boucle DeepAlgebra est un pattern iteratif ou le LLM genere une preuve, Lean la verifie, et le feedback d'erreur est utilise pour corriger. Chaque iteration affine la preuve.

In [6]:
# Simulation de la boucle DeepAlgebra

def deep_algebra_loop(theorem: str, max_iterations: int = 10):
    """
    Boucle iterative d'amelioration de preuve.
    
    1. LLM genere une preuve
    2. Lean verifie
    3. Si erreur -> feedback au LLM
    4. LLM corrige et recommence
    """
    
    history = []
    current_proof = None
    
    for i in range(max_iterations):
        # Generer ou corriger la preuve
        if current_proof is None:
            prompt = f"Genere une preuve Lean 4 pour: {theorem}"
        else:
            prompt = f"""
            Preuve precedente: {current_proof}
            Erreur Lean: {last_error}
            Corrige la preuve.
            """
        
        # Simuler la reponse LLM
        current_proof = f"-- Iteration {i+1}\nby simp"
        
        # Simuler la verification Lean
        lean_result = {"success": i >= 3, "error": "unknown tactic"}
        
        history.append({
            "iteration": i + 1,
            "proof": current_proof,
            "result": lean_result
        })
        
        if lean_result["success"]:
            print(f"Preuve trouvee en {i+1} iterations!")
            return current_proof
        
        last_error = lean_result["error"]
    
    return None

# Demonstration
result = deep_algebra_loop("theorem test : 1 + 1 = 2")

Preuve trouvee en 4 iterations!


### 3.4 APOLLO : Collaboration Automatique LLM-Lean

**APOLLO** (https://arxiv.org/abs/2505.05758) pousse l'automatisation au maximum avec une boucle entierement autonome :

**Caracteristiques** :
- Generation **massive** de candidats de preuve (milliers en parallele)
- **Filtrage** par verification formelle avec Lean
- **Optimisation** par apprentissage sur les succes/echecs
- **Aucune intervention humaine** requise apres lancement

**Resultats** :
- Amelioration de 20-30% sur les benchmarks miniF2F
- Capable de resoudre des theoremes Mathlib non trivaux
- Temps median de resolution : quelques minutes par theoreme

**Principe** : Au lieu de generer une seule preuve et iterer, APOLLO genere des milliers de candidats varies simultanement, les filtre par verification Lean, et utilise les patterns des succes pour ameliorer la generation future.

### 3.5 Comparaison des approches

| Approche | Automatisation | Forces | Faiblesses |
|----------|---------------|--------|------------|
| **Vibe coding** | Faible | Accessibilite, flexibilite | Lent, expertise requise |
| **LeanCopilot** | Moyenne | Temps reel, IDE integre | Modele local limite |
| **DeepAlgebra Loop** | Haute | Apprentissage iteratif | Feedback delays |
| **APOLLO** | Tres haute | Parallelisme massif | Cout computationnel |
| **AlphaProof** | Complete | Performance SOTA | Ressources enormes |

In [7]:
# Architecture conceptuelle APOLLO

class APOLLOConcept:
    """
    APOLLO : Automated Proving with LLM-Lean Optimization
    
    Caracteristiques:
    - Generation parallele massive de preuves candidates
    - Verification formelle systematique
    - Self-play pour l'amelioration
    """
    
    def __init__(self, num_workers: int = 100):
        self.num_workers = num_workers
        self.successful_proofs = []
    
    def prove_massively(self, theorem: str, num_candidates: int = 10000):
        """
        Genere des milliers de candidats en parallele.
        """
        candidates = []
        
        # Phase 1: Generation massive
        for i in range(num_candidates):
            # Variation de temperature et de prompts
            candidate = self.generate_candidate(
                theorem, 
                temperature=0.5 + (i % 10) * 0.1
            )
            candidates.append(candidate)
        
        # Phase 2: Verification parallele
        verified = self.verify_parallel(candidates)
        
        # Phase 3: Retourner les succes
        successes = [c for c in verified if c["valid"]]
        
        return successes
    
    def generate_candidate(self, theorem: str, temperature: float):
        """Genere un candidat de preuve."""
        return f"-- candidate with temp {temperature}"
    
    def verify_parallel(self, candidates):
        """Verifie les candidats en parallele."""
        return [{"proof": c, "valid": True} for c in candidates[:3]]

print("Architecture APOLLO illustree")

Architecture APOLLO illustree


## 4. Prompting Efficace pour Lean

Le prompting pour Lean necessite precision : specifier la version (Lean 4), les imports disponibles, le contexte (hypotheses), et le style de preuve souhaite (termes ou tactiques).

In [8]:
# Templates de prompts efficaces pour Lean

PROMPTS = {
    "basic": """
Ecris une preuve Lean 4 pour le theoreme suivant:

```lean
{theorem}
```

Utilise des tactiques standard (apply, exact, intro, rw, simp).
    """,
    
    "with_context": """
Je travaille dans Lean 4 avec les imports suivants:
{imports}

Voici les hypotheses disponibles:
{hypotheses}

Je dois prouver:
{goal}

Quelle sequence de tactiques dois-je utiliser?
    """,
    
    "iterative": """
Ma preuve actuelle:
```lean
{current_proof}
```

Erreur Lean:
```
{error}
```

Comment corriger cette erreur? Donne la preuve complete corrigee.
    """,
    
    "expert": """
Tu es un expert en Lean 4 et Mathlib4. 

Theoreme a prouver:
{theorem}

Contraintes:
- Utilise les tactiques Mathlib si appropriees (ring, linarith, omega, simp)
- Prefere les preuves courtes et elegantes
- Commente les etapes non triviales

Fournis le code Lean complet.
    """
}

print("Templates de prompts disponibles:")
for name in PROMPTS:
    print(f"  - {name}")

Templates de prompts disponibles:
  - basic
  - with_context
  - iterative
  - expert


### 4.2 Bonnes pratiques

- Inclure le contexte complet (imports, variables, hypotheses)
- Demander des tactiques specifiques plutot que des preuves completes
- Fournir des exemples similaires (few-shot)
- Iterer avec les messages d'erreur Lean

In [9]:
# Bonnes pratiques pour le prompting Lean

BEST_PRACTICES = """
### Prompting efficace pour preuves Lean

1. **Contexte precis**
   - Specifier la version de Lean (4.x)
   - Mentionner les imports disponibles
   - Donner les hypotheses du contexte

2. **But clair**
   - Formuler le theoreme exactement
   - Preciser le type des variables
   - Indiquer si c'est sur Nat, Int, Real, etc.

3. **Contraintes**
   - Tactiques preferees ou interdites
   - Style de preuve (term-mode vs tactic-mode)
   - Longueur souhaitee

4. **Feedback iteratif**
   - Inclure les erreurs Lean exactes
   - Montrer la preuve partielle
   - Demander des corrections specifiques

5. **Exemples similaires**
   - Donner des preuves similaires reussies
   - Montrer le style attendu
   - Few-shot learning ameliore les resultats
"""

print(BEST_PRACTICES)


### Prompting efficace pour preuves Lean

1. **Contexte precis**
   - Specifier la version de Lean (4.x)
   - Mentionner les imports disponibles
   - Donner les hypotheses du contexte

2. **But clair**
   - Formuler le theoreme exactement
   - Preciser le type des variables
   - Indiquer si c'est sur Nat, Int, Real, etc.

3. **Contraintes**
   - Tactiques preferees ou interdites
   - Style de preuve (term-mode vs tactic-mode)
   - Longueur souhaitee

4. **Feedback iteratif**
   - Inclure les erreurs Lean exactes
   - Montrer la preuve partielle
   - Demander des corrections specifiques

5. **Exemples similaires**
   - Donner des preuves similaires reussies
   - Montrer le style attendu
   - Few-shot learning ameliore les resultats



## 5. Integration Pratique avec Python

### 5.1 Utilisation de l'API OpenAI

In [10]:
import os
import sys
from pathlib import Path

# Ajouter le repertoire courant au path pour importer lean_runner
sys.path.insert(0, str(Path.cwd()))

# Charger les variables d'environnement
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("Note: python-dotenv non installe, utilisation des variables systeme uniquement")

def generate_lean_proof(theorem: str, model: str = None) -> str:
    """
    Genere une preuve Lean en utilisant l'API OpenAI.
    
    Args:
        theorem: L'enonce du theoreme a prouver
        model: Le modele a utiliser (defaut: OPENAI_CHAT_MODEL_ID ou gpt-4o)
    
    Returns:
        La preuve generee
    """
    api_key = os.getenv("OPENAI_API_KEY")
    model = model or os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o")
    
    if not api_key or api_key.startswith("sk-..."):
        # Mode simulation si pas de cle API valide
        print("  [Mode simulation - configurez OPENAI_API_KEY dans .env pour utiliser l'API reelle]")
        return f"""-- Preuve simulee (pas de cle API)
theorem simulation_example : True := by trivial"""
    
    # API reelle OpenAI
    try:
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        
        prompt = f"""Tu es un expert en Lean 4. Ecris une preuve pour le theoreme suivant.
Donne UNIQUEMENT le code Lean complet, sans explications.

Theoreme a prouver:
{theorem}

Utilise les tactiques appropriees (rfl, simp, omega, exact, apply, intro, etc.)."""
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Tu es un expert en Lean 4 et Mathlib4."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,  # Basse temperature pour plus de determinisme
            # gpt-5.x et modeles recents utilisent max_completion_tokens
            **({'max_completion_tokens': 500} if model.startswith(('gpt-5', 'o1', 'gpt-4.5')) else {'max_tokens': 500})
        )
        
        proof = response.choices[0].message.content
        # Extraire le code Lean si encapsule dans des backticks
        if "```lean" in proof:
            proof = proof.split("```lean")[1].split("```")[0].strip()
        elif "```" in proof:
            proof = proof.split("```")[1].split("```")[0].strip()
        
        return proof
        
    except ImportError:
        print("  [openai non installe - pip install openai]")
        return "-- Module openai non installe"
    except Exception as e:
        print(f"  [Erreur API: {e}]")
        return f"-- Erreur API: {e}"

# Test
theorem = "theorem add_comm_test (a b : Nat) : a + b = b + a"
print(f"Theoreme: {theorem}")
print(f"\nGeneration de preuve...")
proof = generate_lean_proof(theorem)
print(f"\nPreuve generee:\n{proof}")

Theoreme: theorem add_comm_test (a b : Nat) : a + b = b + a

Generation de preuve...
  [Erreur API: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}]

Preuve generee:
-- Erreur API: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}


### 5.2 Utilisation de l'API Anthropic (Claude)

In [11]:
def generate_lean_proof_claude(theorem: str) -> str:
    """
    Genere une preuve Lean avec Claude (Anthropic).
    """
    api_key = os.getenv("ANTHROPIC_API_KEY")
    
    if not api_key:
        return "-- API Anthropic non configuree"
    
    # Code reel avec Anthropic
    # from anthropic import Anthropic
    # client = Anthropic(api_key=api_key)
    # 
    # message = client.messages.create(
    #     model="claude-3-5-sonnet-20241022",
    #     max_tokens=1024,
    #     messages=[
    #         {"role": "user", "content": f"""
    #             Ecris une preuve Lean 4 pour:
    #             {theorem}
    #             
    #             Donne uniquement le code Lean.
    #         """}
    #     ]
    # )
    # return message.content[0].text
    
    return "-- Simulation Claude"

print("Fonction Claude definie")

Fonction Claude definie


### 5.3 Feedback Loop avec LeanRunner

Le vrai pouvoir de l'integration LLM-Lean vient du **feedback loop** : generer une preuve, la verifier avec Lean, et corriger iterativement si elle echoue.

In [12]:
# Feedback Loop complet avec LeanRunner
# Ce code verifie reellement les preuves generees par le LLM

def prove_with_feedback_loop(
    theorem: str,
    max_iterations: int = 5,
    verbose: bool = True
) -> dict:
    """
    Boucle complete de generation et verification de preuves.
    
    1. Generer une preuve avec OpenAI
    2. Verifier avec LeanRunner
    3. Si echec, utiliser l'erreur pour corriger
    4. Repeter jusqu'a succes ou max_iterations
    
    Args:
        theorem: Le theoreme a prouver (declaration complete)
        max_iterations: Nombre max de tentatives
        verbose: Afficher les details
    
    Returns:
        Dict avec succes, preuve, iterations, historique
    """
    try:
        from lean_runner import LeanRunner
        runner = LeanRunner(timeout=30)
        lean_available = True
    except Exception as e:
        if verbose:
            print(f"[LeanRunner non disponible: {e}]")
        lean_available = False
    
    history = []
    current_proof = None
    last_error = None
    
    for iteration in range(1, max_iterations + 1):
        if verbose:
            print(f"\n--- Iteration {iteration}/{max_iterations} ---")
        
        # Etape 1: Generer ou corriger la preuve
        if current_proof is None:
            # Premiere generation
            if verbose:
                print("Generation initiale...")
            current_proof = generate_lean_proof(theorem)
        else:
            # Correction basee sur l'erreur
            if verbose:
                print(f"Correction basee sur l'erreur: {last_error[:100]}...")
            
            correction_prompt = f"""Le code Lean suivant a une erreur:

```lean
{current_proof}
```

Erreur Lean:
```
{last_error}
```

Corrige le code pour que le theoreme soit prouve correctement.
Donne UNIQUEMENT le code Lean corrige, sans explications."""
            
            api_key = os.getenv("OPENAI_API_KEY")
            if api_key and not api_key.startswith("sk-..."):
                try:
                    from openai import OpenAI
                    client = OpenAI(api_key=api_key)
                    response = client.chat.completions.create(
                        model=os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o"),
                        messages=[
                            {"role": "system", "content": "Tu es un expert Lean 4."},
                            {"role": "user", "content": correction_prompt}
                        ],
                        temperature=0.4,
                        # gpt-5.x et modeles recents utilisent max_completion_tokens
                        **({'max_completion_tokens': 500} if os.getenv('OPENAI_CHAT_MODEL_ID', 'gpt-4o').startswith(('gpt-5', 'o1', 'gpt-4.5')) else {'max_tokens': 500})
                    )
                    current_proof = response.choices[0].message.content
                    if "```lean" in current_proof:
                        current_proof = current_proof.split("```lean")[1].split("```")[0].strip()
                    elif "```" in current_proof:
                        current_proof = current_proof.split("```")[1].split("```")[0].strip()
                except Exception as e:
                    if verbose:
                        print(f"[Erreur API: {e}]")
        
        if verbose:
            print(f"Preuve tentee:\n{current_proof[:200]}...")
        
        # Etape 2: Verifier avec Lean
        if lean_available:
            result = runner.run(current_proof)
            success = result.success
            last_error = result.errors if not success else ""
        else:
            # Simulation si LeanRunner non disponible
            success = iteration >= 3  # Simule succes apres 3 iterations
            last_error = "LeanRunner non disponible - simulation"
        
        history.append({
            "iteration": iteration,
            "proof": current_proof,
            "success": success,
            "error": last_error
        })
        
        if success:
            if verbose:
                print(f"\n[SUCCES] Preuve verifiee en {iteration} iteration(s)!")
            return {
                "success": True,
                "proof": current_proof,
                "iterations": iteration,
                "history": history
            }
        else:
            if verbose:
                print(f"[ECHEC] Erreur: {last_error[:150]}...")
    
    if verbose:
        print(f"\n[ECHEC] Pas de preuve trouvee apres {max_iterations} iterations")
    
    return {
        "success": False,
        "proof": current_proof,
        "iterations": max_iterations,
        "history": history
    }

# Demonstration du feedback loop
print("=" * 60)
print("DEMONSTRATION: Feedback Loop LLM + Lean")
print("=" * 60)

# Theoreme simple pour la demonstration
demo_theorem = "theorem demo_add_zero (n : Nat) : n + 0 = n := by rfl"
result = prove_with_feedback_loop(demo_theorem, max_iterations=3, verbose=True)

print("\n" + "=" * 60)
print(f"Resultat final: {'SUCCES' if result['success'] else 'ECHEC'}")
print(f"Iterations: {result['iterations']}")

DEMONSTRATION: Feedback Loop LLM + Lean

--- Iteration 1/3 ---
Generation initiale...
  [Erreur API: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}]
Preuve tentee:
-- Erreur API: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'p...
[ECHEC] Erreur: Timeout after 30 seconds...

--- Iteration 2/3 ---
Correction basee sur l'erreur: Timeout after 30 seconds...
[Erreur API: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}]
Preuve tentee:
-- Erreur API: Error code: 400 - {'error': {

## 6. Problemes d'Erdos : Benchmark pour LLMs

Les **problemes d'Erdos** sont devenus un benchmark populaire pour evaluer les systemes de theorem proving assistes par LLM.

### 6.1 Problemes resolus par IA

In [13]:
# Problemes d'Erdos resolus par des systemes IA (mise a jour janvier 2026)

ERDOS_SOLVED = {
    "379": {
        "domaine": "Graphes extremaux",
        "ouvert_depuis": "~30 ans",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Conjecture sur le nombre chromatique de graphes"
    },
    "987": {
        "domaine": "Combinatoire additive",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Sommes de sous-ensembles dans Z"
    },
    "730": {
        "domaine": "Theorie des nombres",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Nombres premiers jumeaux generalises"
    },
    "198": {
        "domaine": "Ensembles",
        "resolu_par": "DeepSeek-Prover",
        "annee": "2025",
        "description": "Intersections d'ensembles disjoints"
    },
    "124_variant": {
        "domaine": "Graphes aleatoires",
        "ouvert_depuis": "~30 ans",
        "resolu_par": "Harmonic Aristotle",
        "temps": "6 heures",
        "annee": "2025",
        "description": "Proprietes des graphes aleatoires"
    }
}

# Explosion des resolutions depuis Noel 2025
CHRISTMAS_ERDOS_WAVE = """
Depuis Noel 2025, une vague de resolutions a frappe la communaute:

- 15+ problemes Erdos resolus par des systemes IA en quelques semaines
- Harmonic Aristotle a obtenu la medaille d'or IMO 2025
- Terry Tao a formalise des resultats en theorie analytique des nombres avec Mathlib
- DeepSeek, Google, et Harmonic en competition pour resoudre des problemes ouverts

Cette acceleration est due a:
1. Meilleurs modeles de base (Gemini 2.0, GPT-5, Claude 3.5)
2. Fine-tuning specifique sur preuves Lean
3. Mathlib4 avec 4M+ lignes de mathematiques formalisees
4. Techniques de recherche ameliorees (MCTS, beam search)
"""

print("Problemes d'Erdos resolus par IA:")
print("=" * 50)
for num, info in ERDOS_SOLVED.items():
    print(f"\nErdos #{num}:")
    for key, value in info.items():
        print(f"  {key}: {value}")

print("\n" + "=" * 50)
print(CHRISTMAS_ERDOS_WAVE)

Problemes d'Erdos resolus par IA:

Erdos #379:
  domaine: Graphes extremaux
  ouvert_depuis: ~30 ans
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Conjecture sur le nombre chromatique de graphes

Erdos #987:
  domaine: Combinatoire additive
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Sommes de sous-ensembles dans Z

Erdos #730:
  domaine: Theorie des nombres
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Nombres premiers jumeaux generalises

Erdos #198:
  domaine: Ensembles
  resolu_par: DeepSeek-Prover
  annee: 2025
  description: Intersections d'ensembles disjoints

Erdos #124_variant:
  domaine: Graphes aleatoires
  ouvert_depuis: ~30 ans
  resolu_par: Harmonic Aristotle
  temps: 6 heures
  annee: 2025
  description: Proprietes des graphes aleatoires


Depuis Noel 2025, une vague de resolutions a frappe la communaute:

- 15+ problemes Erdos resolus par des systemes IA en quelques semaines
- Harmonic Aristotle a obtenu la medaille d'or IMO 2025

## 7. Exercices Pratiques

### Exercice 1 : Creer un prompt pour une preuve simple

In [14]:
# Completez ce prompt pour demander une preuve de la commutativite de la multiplication

EXERCISE_PROMPT = """
# Votre prompt ici
# Objectif: obtenir une preuve de (a * b = b * a) en Lean 4
"""

# Solution:
SOLUTION_PROMPT = """
Tu es un expert en Lean 4.

Ecris une preuve pour le theoreme suivant:

```lean
theorem mul_comm (a b : Nat) : a * b = b * a
```

Utilise la tactique appropriee de la bibliotheque standard.
Donne uniquement le code Lean complet.
"""

print("Exercice 1: Creer un prompt")
print("Voir SOLUTION_PROMPT pour la solution")

Exercice 1: Creer un prompt
Voir SOLUTION_PROMPT pour la solution


### Exercice 2 : Implementer une boucle de correction

In [15]:
def correction_loop(theorem: str, initial_proof: str, errors: list):
    """
    Implemente une boucle de correction iterative.
    
    Args:
        theorem: Le theoreme a prouver
        initial_proof: La preuve initiale (avec erreurs)
        errors: Liste d'erreurs Lean
    
    Returns:
        Le prompt pour corriger la preuve
    """
    # Votre code ici
    pass

# Solution:
def correction_loop_solution(theorem: str, initial_proof: str, errors: list) -> str:
    error_text = "\n".join(errors)
    return f"""
La preuve suivante contient des erreurs:

Theoreme: {theorem}

Preuve actuelle:
```lean
{initial_proof}
```

Erreurs Lean:
```
{error_text}
```

Corrige ces erreurs et fournis la preuve complete et correcte.
    """

# Test
test_prompt = correction_loop_solution(
    "theorem test : 1 + 1 = 2",
    "by rfl",
    ["type mismatch"]
)
print(test_prompt[:200] + "...")


La preuve suivante contient des erreurs:

Theoreme: theorem test : 1 + 1 = 2

Preuve actuelle:
```lean
by rfl
```

Erreurs Lean:
```
type mismatch
```

Corrige ces erreurs et fournis la preuve comple...


## Resume

### Points cles

| Systeme | Approche | Force | Publication |
|---------|----------|-------|-------------|
| **LeanCopilot** | Suggestions temps reel | Integration IDE | NeurIPS 2025 |
| **LeanProgress** | Prediction progression | Guidage recherche | TMLR 2025 |
| **LeanAgent** | Lifelong learning | Adaptation | ICLR 2025 |
| **AlphaProof** | RL + generation massive | Theoremes difficiles | Nature 2025 |
| **APOLLO** | Automatisation complete | Scalabilite | arXiv 2505 |
| **Harmonic Aristotle** | Decomposition + search | Problemes ouverts | IMO 2025 Gold |

### Tactiques de prompting

1. **Contexte precis** : version Lean 4, imports, hypotheses
2. **But clairement formule** : types explicites, theoreme exact
3. **Feedback erreurs Lean** : messages d'erreur complets
4. **Exemples similaires** : few-shot avec preuves reussies
5. **Iterations successives** : corriger et re-soumettre

### Ressources et liens

| Ressource | URL |
|-----------|-----|
| LeanDojo (ecosysteme) | https://leandojo.org |
| LeanCopilot (GitHub) | https://github.com/lean-dojo/LeanCopilot |
| AlphaProof (Nature) | https://www.nature.com/articles/s41586-025-08589-7 |
| APOLLO (arXiv) | https://arxiv.org/abs/2505.05758 |
| Mathlib4 | https://github.com/leanprover-community/mathlib4 |
| Loogle (recherche) | https://loogle.lean-lang.org |
| Moogle (semantic) | https://www.moogle.ai |
| Xena Project (formalisations) | https://xenaproject.wordpress.com |

### Prochaine etape

Dans le notebook **Lean-8-Agentic-Proving**, nous construirons un **systeme multi-agents** capable de prouver des theoremes de maniere autonome, en orchestrant :
- **Agent de recherche** : Trouve des lemmes pertinents dans Mathlib
- **Agent de generation** : Propose des tactiques et preuves
- **Agent de verification** : Valide avec Lean et fournit du feedback
- **Orchestrateur** : Coordonne les agents avec Semantic Kernel

---

*Notebook base sur les percees IA 2024-2026 en theorem proving (AlphaProof/Nature 2025, APOLLO, LeanCopilot/NeurIPS 2025, LeanAgent/ICLR 2025, LeanProgress/TMLR 2025)*

---

**Navigation** : [← Lean-6-Mathlib-Essentials](Lean-6-Mathlib-Essentials.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-8-Agentic-Proving →](Lean-8-Agentic-Proving.ipynb)